# Reinforcement Learning - Summary

Lecture summer 2024 by *Prof. Matthias Niepert* at university of Stuttgart


## Table of Contents
1. [Introduction](#introduction)
2. [Markov Decision Processes](#markov-decision-processes)
3. [Monte Carlo](#monte-carlo)
4. [Temporal Difference](#temporal-difference)

## Introduction

### RL - Problem
* General framework for decision making
* Agent -> max reward (long-term)

<img src="slides/images/RL_Context.png" width="40%">
<img src="slides/images/RL_Context_ML.png" width="40%">

#### RL - Cycle
<img src="slides/images/RL_Cycle.png" width="80%">

with state $S_t$, action $A_t$, reward $R_t$ and history $H_t$

$$ \begin{align} H_t = S_0, A_0, R_1, S_1, A_1, R_2, ... S_{t-1}, A_{t-1}, R_t \end{align} $$


#### Markov Property

A state $S_t$ is Markov if and only if
$$ \begin{align} \text{Pr}\{S_{t+1}\} = \text{Pr}\{ S_{t+1} | S_1, ..., S_t \} \end{align} $$

e.i. the future is independent of the past given the present.

#### Markov - Types

* Agent observes Markov state - Markov Decision Process (MDP)

* observes indirectly $\to$ Partially Obeservable MDP (POMDP) 


### RL - Agent

* Policy $\pi$ - agent's behavior (det. or stoch.)
* Value function $V(s)$ - expected return from state $s$
* Action-value function $Q(s,a)$ - expected return from state $s$ and action $a$
* Model - agent's representation of the environment

The true model aka transition function is given by
$$ \begin{align} p(s',r|s,a) = \text{Pr}\{ S_{t+1} = s', R_{t+1} = r| S_t = s, A_t = a\} \end{align} $$

#### RL - Flavors

```mermaid
mindmap
  root((RL))
    model-based
      id[test]
    model-free
      value-based
      policy-based
      actor-critic
    imitation learning
```
Exploitation vs Exploration actions, where the reward follows a probability distribution -> max. expected reward

#### Bandits - Tabular solution methods

If state and action spaces are small enough
* find exact solution
  * optimal V
  * optimal $\pi$

Code example can be found [here](exercises/exercise_01/ex01-bandits.py)

#### Greedy and Eplsilon-Greedy

* Greedy for $\epsilon = 0$ which is the percentage of doing an other (random or softmax) action instead of the believed best action

$$ \begin{align} A_t = \text{argmax}_a Q_t(a) \end{align} $$

* Finetuning $\varepsilon$
  * reward variance is small, e.g. zero
  * reward variance is large
  * task is non-stationary

$$ \begin{align} \pi_t(a) = \frac{e^{\frac{Q_t(a)}{\tau}}}{\sum_{a' = 1}^k e^{\frac{Q_t(a')}{\tau}}} \end{align} $$

Gibbs or Boltzmann distribution with Temperature $\tau$ which is continuous $0$ and random $\infty$.

Incremental equation
$$ \begin{align} Q_{n+1} = Q_n + \underbrace{\frac{1}{n}}_{\text{generally } \alpha} [R_n - Q_n] \end{align} $$

## Markov Decision Processes

### Definitions

<img src="slides/images/MDP_Defi.png" width="80%">


#### Goal

Cumulative reward with discount factor

$$ \begin{align} G_t &= \sum_{i=0}^\infty \gamma^i R_{t+i+1} \\
&= R_{t+1} + \gamma G_{t+1} \end{align} $$

with $\gamma \in [0,1]$

#### Transition Function

$$ \begin{align} p(s'|s,a) &= \text{Pr}\{ S_{t+1} = s'|S_t=s, A_t = a \} \\ &= \sum_{r \in \mathcal{R}} p(s',r|s,a) \end{align} $$

#### Reward function

immediate reward

$$ \begin{align} r(s,a,s') &= \mathbb{E}\left[R_{t+1}|S_t = s, A_t = a, S_{t+1} = s'\right] \\ &= \sum_{r \in \mathcal{R}} r \frac{p(s',r|s,a)}{p(s'|s,a)} \end{align} $$

Note - typically we assume there is a single reward for each $s,a,s'$ and drop the $\mathbb{E}$.

##### Collection rewards

$$ \begin{align} r(s,a) &= \sum_{r \in \mathcal{R}} r \sum_{s'\in \mathcal{S}} p(s',r|s,a) \\ r(s) &= ...
 \end{align} $$

 #### Transition Graph

```mermaid
graph LR
    id0((low)) --> id01[recharge]
    id01 --$$1,\:0$$--> id1((high))


    id0 --$$1, r_{wait}$$--> id02[search]
    id02 --$$\beta, r_{search}$$--> id0
    id02 --$$1-\beta, -3$$--> id1

    id0 --> id03[wait]
    id03 --$$1,\: r_{wait}$$--> id0


    id1 --> id11[search]
    id11 --$$\alpha, r_{search}$$--> id1
    id11 --$$1-\alpha, \: r_{search}$$--> id0

    id1 --> id12[wait]
    id12 --$$1, r_{search}$$--> id1

```

#### Bellman Equation

Recursive equation which is stationary for optimum

##### Value Function
$$ \begin{align} v_\pi(s) &= \sum_{a} \pi(a|s) \sum_{s',r} p(s',r|s,a)\left[r + \gamma v_\pi(s')\right] \qquad \forall \: s \in \mathcal{S} \\
v_*(s) &= \max_a \sum_{s',r} p(s',r|s,a)\left[r + \gamma v_*(s')\right] 
\end{align}$$
##### Action-Value Function
$$\begin{align}
q_\pi(s,a) &= \sum_{s',r} p(s',r|s,a)\left[r + \gamma \sum_{a'} \pi(a'|s') q_\pi(s',a')\right] \\
q_*(s,a) &= \sum_{s',r} p(s',r|s,a)\left[r + \gamma 
\max_{a'} q_*(s',a')\right] \end{align}$$

##### Relation toward each other
$$ \begin{align}
q_\pi(s,a) &= \sum_{s',r} p(s',r|s,a)\left[r + \gamma v_\pi(s') \right]
\end{align}$$

since $v_\pi(s) = \sum_{a} \pi(a|s) q_\pi(s,a)$


#### Bellman: Matrix Form

$$ \begin{align} v = (I - \gamma\mathcal{P})^{-1}\mathcal{R} \end{align}$$

where $\mathcal{P}$ is the transition matrix 
$$ \begin{align}\mathcal{P} = \begin{pmatrix} p_{11} & \dots & p_{1n}\\
\vdots & \ddots & \vdots \\
p_{n1} & \dots & p_{nn} \end{pmatrix} \end{align} $$
and $\mathcal{R}$ the reward matrix


#### Optimal Policy

$$ \begin{align} v_*(s) &= \max_\pi v_\pi(s) \quad \forall \: s \in \mathcal{S} \\
q_*(s,a) &= \max_\pi \underbrace{\mathbb{E}_\pi \left[ R_{t+1} + \gamma v_*(S_{t+1})|S_t = s, A_t = t \right]}_{q_\pi(s,a)} \end{align} $$





In [8]:
# Bellman Equation solved for slippery gridworld
import numpy as np
import gym
from itertools import product

custom_map3x3 = [
    'SFF',
    'FFF',
    'FHG',
]
env = gym.make("FrozenLake-v0", desc=custom_map3x3)


# Init some useful variables:
n_states = env.observation_space.n
n_actions = env.action_space.n

r = np.zeros(n_states) # the r vector is zero everywhere except for the goal state (last state)
r[-1] = 1.

gamma = 0.8


""" This is a helper function that returns the transition probability matrix P for a policy """
def trans_matrix_for_policy(policy):
    s_term = terminals()
    possible_states = [s for s in range(n_states) if s not in s_term]
    transitions = np.zeros((n_states, n_states))
    for s in possible_states:
        probs = env.P[s][policy[s]]
        for el in probs:
            transitions[s, el[1]] += el[0]
    return transitions


""" This is a helper function that returns terminal states """
def terminals():
    terms = []
    for s in range(n_states):
        # terminal is when we end with probability 1 in terminal:
        if env.P[s][0][0][0] == 1.0 and env.P[s][0][0][3] == True:
            terms.append(s)
    return terms


def value_policy(policy):
    P = trans_matrix_for_policy(policy)
    P[-1] = 0
    # TODO: calculate and return v
    # (P, r and gamma already given)
    v = np.linalg.solve(np.eye(n_states) - gamma * P, r)
    return v


def bruteforce_policies():
    terms = terminals()
    number_of_non_terminal_states = n_states - len(terms)
    optimalpolicies = []
    all_policies = []
    number_of_policies = n_actions**number_of_non_terminal_states

    for i in range(number_of_policies):
        # Initialize an empty list to store the current policy
        policy = []
        # For each state, determine the corresponding action
        # We do this by dividing by the appropriate power of n_actions
        # and taking the modulus with n_actions
        value = i
        for j in range(number_of_non_terminal_states):
            # Find the action for this state
            action = value % n_actions  # Get the remainder to determine the action
            policy.append(action)

            # Update the value to get the next action
            value = value // n_actions

        all_policies.append(policy)

    # fill spots of terminal states with random action (it doesnt matter)
    for term in terms:
        for policy in all_policies:
            policy.insert(term, 1)

    all_policies = np.array(all_policies)

    values = np.array([value_policy(policy) for policy in all_policies])
    values_sum = values.sum(axis=1)
    optimalvalue_index = np.argmax(values_sum)
    optimalvalue = values[optimalvalue_index]

    matches = [np.array_equal(optimalvalue, sub_array) for sub_array in values]
    optimalpolicies = all_policies[matches]
    

    print("Optimal value function:")
    print(optimalvalue)
    print("number optimal policies:")
    print(len(optimalpolicies))
    print("optimal policies:")
    print(np.array(optimalpolicies))

    return optimalpolicies


def main():
    # print the environment
    print("current environment: ")
    env.reset()
    env.render()
    print("")

    # Here a policy is just an array with the action for a state as element
    policy_left = np.zeros(n_states, dtype=int)  # 0 for all states
    policy_right = np.ones(n_states, dtype=int) * 2  # 2 for all states

    # Value functions:
    print("Value function for policy_left (always going left):")
    print(value_policy(policy_left))
    print("Value function for policy_right (always going right):")
    print(value_policy(policy_right))

    optimalpolicies = bruteforce_policies()


    # This code can be used to "rollout" a policy in the environment:
    print("rollout policy:")
    maxiter = 100
    terminated = False
    for i in range(10):
        state, _ = env.reset()
        for j in range(maxiter):
            
            new_state = env.step(optimalpolicies[0][state])
            if new_state in terminals():
                terminated = True
            env.render()
            state=new_state
            if terminated:
                print("Finished episode")
                break


if __name__ == "__main__":
    main()


current environment: 

SFF
FFF
FHG

Value function for policy_left (always going left):
[0.         0.         0.10738255 0.         0.         0.29530201
 0.         0.         1.        ]
Value function for policy_right (always going right):
[0.08280355 0.15491253 0.26229508 0.07279724 0.16371438 0.45901639
 0.02647172 0.         1.        ]
Optimal value function:
[0.09951342 0.16642762 0.26229508 0.10723429 0.19538088 0.45901639
 0.06127674 0.         1.        ]
number optimal policies:
2
optimal policies:
[[1 2 2 3 3 2 0 1 1]
 [2 2 2 3 3 2 0 1 1]]
rollout policy:
  (Down)
SFF
FFF
FHG


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Policy Evaluation

### Iterative Policy Evaluation

```python
for s in states:
    v_temp = V[s]
    V[s] = np.sum([pi[s,a] * np.sum([p[s,a,s1] * (r[s,a,s1] + gamma * V[s1]) for s1 in states]) for a in actions])
    delta = max(delta, np.abs(v_temp - V[s]))
```
until $\Delta < \theta$.

<img src="notes/image/algorithms/policy_iteration.png" width="50%">


#### Contraction

Using the infinity norm, we can use the bellman equation as a contraction

$$ \begin{align} (T^\pi v)(s) = \sum_a \pi(a|s) \sum_{s',r} p(s',r|s,a)\left[r + \gamma v(s') \right] \end{align} $$

proof involves property of maximum norm ($\infty$-norm) see [here](slides/Lecture%2003.pdf)


#### Asynchronous DP

* In-place
* Prioritized sweeping (e.g. Bellman error $|v_{bel} - v|) e.g. via queue
* real-time



## Monte Carlo

Estimation of black box $f$ w.r.t. some distribution which satisfies

* Samples i.i.d.
* $f$ encodes environment and returns sequence of experience

### MC in RL

* **Learn** value function $v()$ from *experience* (env or sim)
* **Discover** optimal policies
* **Blackbox view** ($p(s'|s,a)$ and $r(s,a,s')$ isnt required)

<img src="notes/image/blackbox.png" width="50%">

#### MC principle

* Divide into (terminating!) episodes
* estimate $v(\cdot)$ and update each **episode** (instead of each step like DP)
* TODO: cant use $v(\cdot)$ to improve $\pi$
* $v(\cdot)$ is approx. for $s, \pi$ pair
    * first visit
    * every visit





#### MC properties

* Independent estimate for $v(\cdot)$
* Compute time is independent of $|\mathcal{S}|$
* Possible to focus on relevant states and ignore the value of others

Need to estimate 
$$ \begin{align} \pi'(s) = \argmax_a q_\pi(s,a) \end{align} $$

And similar
$$ \begin{align} q_\pi(s,a) = \mathbb{E}_\pi \left[ G_t|S_t = s, A_t = a \right] \end{align} $$
by averaging returns following first visit to that state-action pair [Warning p.20](slides/Lecture%2004.pdf)

#### MC control

<img src="notes/image/MC_control.png" width="50%">

* converges




### On-policy
* learn about policy currently used to generate experience

<img src="notes/image/algorithms/MC/First-visit_MC_control.png" width="50%">


### Off-policy
* evaluate and improve a policy that is different from the one used for genereating episodes
* *target* policy $\pi$ and *behavior* policy $\mu$
* it holds that $\mu$ generates behavior which covers $\pi$ 
$$ \begin{align} \pi(a|s) > 0 \implies \mu(a|s) > 0 \: \forall_{s,a} \end{align} $$
*compute MC estimates from the trajectories generated by $\mu$ but make adjustments such that we obtain estimates compatible with $\pi$*




### Importance Sampling

* Target distribution $p(x)$
* proposal distribution $q(x)$

$$ \begin{align} \mathbb{E}_{p(x)}[f(x)] &= \int f(x) p(x) \mathrm{d}x = \int f(x) \frac{p(x)}{q(x)} q(x) \mathrm{d}x = \mathbb{E}_{q(x)} \left[ f(x) \frac{p(x)}{q(x)} \right] \\
&\approx \frac{1}{L} \sum_{l=1}^L f(x_l) \underbrace{\frac{p(x_l)}{q(x_l)}}_{w_l}, \quad \text{with samples} \: x_l \overset{i.i.d.}{\sim} q(x) \end{align} $$

See further [here](slides/Lecture%2004.pdf)

## Temporal Difference
* combines **sampling** [MC] with **bootstrapping** (update based in part on existing estimate) [DP] see page 9 [here](slides/Lecture%2005.pdf)

### Incremental Monte Carlo

$$ \begin{align} V(s) \leftarrow V(s) + \underbrace{\frac{1}{n(s) + 1}}_{\alpha} [\underbrace{G_t}_{\text{MC target}} - V(s)] \end{align} $$ 
where $\alpha$ could be given as a relation with respect to the number of first visits of states $s$ 

Simplest temporal difference update TD(0)
$$ target = R_{t+1} + \gamma V(S_{t+1})$$



### Comparison

| TD | MC |
| --- | --- |
| learn before terminal | unbiased |
| non-episodic or episodic task | - |
| lower variance | - |
| bias - TD updates exploit the Markov property| |

both converge but these value function are potentially different. See example [here](slides/Lecture%2005.pdf)

<img src="notes/image/comparison_square.png" width="60%">
<!-- <img src="notes/image/TD_MC_DP.png" width="40%"> -->

### TD for control
estimate $Q$ instead of $V$

$$ Q(S_t, A+t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1}) - Q(S_t, A_t)] $$

* change $\pi$ toward greediness wrt $Q_\pi$
* use soft policies, e.g. $\varepsilon$-greedy
* converges to $Q_*$

#### SARSA (on-policy)
<img src="notes/image/algorithms/SARSA.png" width="80%">

#### Q-learning (off-policy)
* Target policy $\pi$ is greedy $$\pi(a) = \argmax_a Q(s,a)$$
* Behavior policy $\mu$ is soft plicy, e.g. $\varepsilon-$ greedy
<img src="notes/image/algorithms/Qlearning.png" width="80%">

#### Comparison of Q-learning and SARSA

<img src="notes/image/Sarsa_vs_Q.png" width="80%">

#### Extensions

##### Expected SARSA

Both on- and off-policy


##### Maximization bias

* positive maximization bias

<img src="notes/image/examples/max_bias.png" width="30%">

##### Double Q-learning

* Divide into $Q_1$ and $Q_2$ 



## Models & Planning

### Model
* Anything the agent can use like an environment
    * Distribution model (ideal)
    * Sample model (approx.) 

### Planning
* Uses model, e.g. $\text{model} \overset{\text{planning}}{\to}\text{policy}$
* Types
  * state-space planning
    * search through the state space
    * e.g. classical DP, heustic search
  * plan-space (IGNORE, not done in lecture)


<img src="notes/image/RL_cycle.png" width="50%">


| Direct | Indirect |
| --- | --- |
| simpler | fuller use of experience |
| not affected by bad model | better policy with fewer env it |

* can occur *simultaneously* and in *parallel*



### Dyna architecture

<img src="notes/image/algorithms/DynaQ.png" width="80%">

#### DYna-Q+

Uses exploration bonus

$$ \begin{align} R + \kappa \sqrt{\tau} \end{align} $$


### Prioritized sweeping

* which $s$ or $(s,a)$ is nice in planning
  * maintain a queue $(s,a)$ who change a lot
  * when a new backup occurs insert predecessors according to their priorities
  * always perform backups from first in queue

  <img src="notes/image/algorithms/PrioritizedSweeping.png" width="80%">
  
   


### Simulation based search
* Planning after encountering a new state $S_t$ (instead of imroving overall policy)

#### Forward search
* build a search tree with the current state $s_t$

<img src="notes/image/forward_search.png" width="50%">

#### Rollout

TODO: simulate from this point onwards, the model therefore is

$$ \begin{align} \{s_t^k, A_t^k, R_{t+1}^k,\dots, S_T^k\}_{k=1}^K \sim \mathcal{M}_v \end{align} $$

Then we can apply model-free RL to simulate episode, e.g
* MC control $\to$ [MC search](#mc-search)
* SARSA $\to$ TD search

#### MC search

##### Simple

* Model $\mathcal{M}_v$, policy $\pi$
* For each action $a \in \mathcal{A}$
$$ \begin{align} \{s_t^k, a_t, R_{t+1}^k, S_{t+1}^k, A_{t+1}^k \dots, S_T^k\}_{k=1}^K \sim \mathcal{M}_v \end{align} $$

evaluate by mean return

$$ \begin{align} Q(s_t, a_t) = \frac{1}{N} \sum_{k=1}^K G_t \overset{P}{\to} q_\pi(s_t, a)  \end{align} $$

$$ a_t = \argmax_{a \in \mathcal{A}} Q(s_t, a)$$


##### Tree

* Selection
* Expansion
* Simulation
  * Tree policy (improves)
  * Rollout policy (fixed)
* Backup


<img src="notes/image/MC_tree_search.png" width="50%">

